# 04 - Visualisation des résultats et comparaison QSVM vs SVM

Ce notebook charge les modèles entraînés, évalue leurs performances,
utilise `ModelEvaluator` pour générer les figures, et assemble un résumé
comparatif pour le rapport final.


In [ ]:
from pathlib import Path
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

from src.pipeline import AudioQSVMpipeline
from src.evaluation import ModelEvaluator
from src.models.quantum_svm import QuantumSVM

features_path = Path('results/features/extracted_features.h5')
results_dir = Path('results/evaluations')
features_path, results_dir


ModuleNotFoundError: No module named 'src'

In [ ]:
if features_path.exists():
    with h5py.File(features_path, 'r') as f:
        X = np.array(f['X'])
        y = np.array(f['y'])
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    pipeline = AudioQSVMpipeline(config_path='config/paths.yaml')
    svm = pipeline._load_svm_or_raise()
    y_pred_svm = svm.predict(X_test)
    y_proba_svm = svm.predict_proba(X_test)

    evaluator = ModelEvaluator(results_dir=results_dir)
    svm_res = evaluator.evaluate_model(
        y_true=y_test, y_pred=y_pred_svm, y_proba=y_proba_svm, model_name='svm_rbf'
    )
    svm_res.metrics


In [ ]:
# Évaluation QSVM si le modèle existe
qsvm_path = Path('models/qsvm_model.joblib')
if features_path.exists() and qsvm_path.exists():
    qsvm = QuantumSVM.load(qsvm_path)
    y_pred_q = qsvm.predict(X_test)
    y_proba_q = qsvm.predict_proba(X_test)

    qsvm_res = evaluator.evaluate_model(
        y_true=y_test, y_pred=y_pred_q, y_proba=y_proba_q, model_name='qsvm'
    )
    qsvm_res.metrics
